<a href="https://colab.research.google.com/github/arjonnill07/Change-Detection-in-paired-images/blob/main/19Nov24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Mount Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
# Install YOLOv8 dependencies
!pip install ultralytics

# Import necessary libraries
import os
import cv2
import numpy as np
from ultralytics import YOLO

# Dataset paths
train_image_A_path = "/content/drive/MyDrive/LEVIR-CD+/train/A"
train_image_B_path = "/content/drive/MyDrive/LEVIR-CD+/train/B"
train_label_path = "/content/drive/MyDrive/LEVIR-CD+/train/label"
test_image_A_path = "/content/drive/MyDrive/LEVIR-CD+/test/A"
test_image_B_path = "/content/drive/MyDrive/LEVIR-CD+/test/B"
test_label_path = "/content/drive/MyDrive/LEVIR-CD+/test/label"

# Paths to save processed YOLO dataset
train_save_path = "/content/yolo_train"
test_save_path = "/content/yolo_test"

os.makedirs(train_save_path + "/images", exist_ok=True)
os.makedirs(train_save_path + "/labels", exist_ok=True)
os.makedirs(test_save_path + "/images", exist_ok=True)
os.makedirs(test_save_path + "/labels", exist_ok=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.2/887.2 kB 15.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:


# Function to convert binary mask to YOLO format
def mask_to_yolo(mask):
    yolo_annotations = []
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    h, w = mask.shape
    for contour in contours:
        x, y, w_box, h_box = cv2.boundingRect(contour)
        xc = (x + w_box / 2) / w
        yc = (y + h_box / 2) / h
        bw = w_box / w
        bh = h_box / h
        yolo_annotations.append(f"0 {xc} {yc} {bw} {bh}")
    return yolo_annotations

# Function to process dataset and save in YOLO format
def prepare_dataset(image_A_path, image_B_path, label_path, save_path):
    img_names = sorted(os.listdir(image_A_path))
    for img_name in img_names:
        img_A = cv2.imread(os.path.join(image_A_path, img_name))
        img_B = cv2.imread(os.path.join(image_B_path, img_name))
        label = cv2.imread(os.path.join(label_path, img_name), cv2.IMREAD_GRAYSCALE)

        # Combine images side by side
        combined_image = np.concatenate((img_A, img_B), axis=1)

        # Convert mask to YOLO annotations
        annotations = mask_to_yolo(label)

        # Save combined image and annotations
        cv2.imwrite(os.path.join(save_path, "images", img_name), combined_image)
        with open(os.path.join(save_path, "labels", img_name.split('.')[0] + ".txt"), "w") as f:
            f.write("\n".join(annotations))

# Process train and test datasets
prepare_dataset(train_image_A_path, train_image_B_path, train_label_path, train_save_path)
prepare_dataset(test_image_A_path, test_image_B_path, test_label_path, test_save_path)

# Create YOLOv8 configuration YAML
yaml_content = f"""
path: /content  # Base path for the dataset
train: {train_save_path}/images  # Path to training images
val: {test_save_path}/images    # Path to validation images
names:
  0: change  # Class name for change detection
"""

yaml_path = "/content/change_detection.yaml"
with open(yaml_path, "w") as yaml_file:
    yaml_file.write(yaml_content)

# Load YOLOv8 model and train
model = YOLO('yolov8n.pt')  # Load pre-trained YOLOv8 nano model
model.train(
    data=yaml_path,  # Path to YAML configuration file
    epochs=10,  # Number of epochs
    batch=16,  # Batch size
    imgsz=640,  # Image size
    project="change_detection_project",  # Project name
    name="yolo_change_detection"  # Experiment name
)

# Validate the model on the test dataset
metrics = model.val()

# Inference on a test image
test_image_path = os.path.join(test_save_path, "images", "train_638.png")  # Replace with an actual test image path
results = model(test_image_path)
results.save()  # Save results with bounding boxes

# Display the results
from IPython.display import Image, display
output_image_path = results.files[0]
display(Image(filename=output_image_path))


100%|██████████| 6.25M/6.25M [00:00<00:00, 69.0MB/s]


Ultralytics 8.3.33 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/change_detection.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=change_detection_project, name=yolo_change_detection, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_la

100%|██████████| 755k/755k [00:00<00:00, 14.4MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/yolo_train/labels... 637 images, 65 backgrounds, 0 corrupt: 100%|██████████| 637/637 [00:19<00:00, 33.39it/s]

...

val: Scanning /content/yolo_test/labels.cache... 348 images, 14 backgrounds, 0 corrupt: 100%|██████████| 348/348 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   5%|▍         | 1/22 [00:04<01:30,  4.33s/it]

WARNING ⚠️ NMS time limit 2.800s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [01:39<00:00,  4.52s/it]


                   all        348      11099    0.00992     0.0933     0.0131    0.00331
Speed: 1.3ms preprocess, 149.3ms inference, 0.0ms loss, 54.8ms postprocess per image
Results saved to change_detection_project/yolo_change_detection2

image 1/1 /content/yolo_test/images/train_638.png: 320x640 (no detections), 128.9ms
Speed: 3.0ms preprocess, 128.9ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)


AttributeError: 'list' object has no attribute 'save'